# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
!pip install hvplot geoviews
# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,28.92,54,40,4.12,US,1724609352
1,1,olonkinbyen,70.9221,-8.7187,3.40,96,100,11.56,SJ,1724609366
2,2,cabedelo,-6.9811,-34.8339,28.37,71,38,8.54,BR,1724609094
3,3,arar,30.9753,41.0381,38.01,13,0,0.00,SA,1724609369
4,4,rafah,31.2870,34.2595,28.38,67,0,4.19,PS,1724609370


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:


# Configure the map plot
city_data_df_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    scale = 1)

# Display the map
city_data_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:

#A max temperature lower than 27 degrees but higher than 21
ideal_temp_df = city_data_df.loc[(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]<26) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] ==0), :]
ideal_temp_df



#Narrow down cities that fit criteria and drop any results with null values
ideal_temp_df.dropna()


# Display sample data
ideal_temp_df.head(11)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
124,124,colonia,50.9333,6.9500,21.65,50,0,1.03,DE,1724609446
157,157,newman,37.3138,-121.0208,24.46,49,0,4.02,US,1724609562
241,241,laguna,38.4210,-121.4238,24.02,49,0,4.12,US,1724609670
259,259,toliara,-23.3500,43.6667,21.26,67,0,2.26,MG,1724609691
368,368,lewiston orchards,46.3804,-116.9754,22.50,43,0,3.09,US,1724609825
370,370,pacifica,37.6138,-122.4869,22.73,75,0,2.06,US,1724609827
384,384,salinas,36.6777,-121.6555,24.90,50,0,2.06,US,1724609846
465,465,pacific grove,36.6177,-121.9166,24.67,68,0,2.06,US,1724609954
495,495,avenal,36.0041,-120.1290,23.09,36,0,3.07,US,1724609992
520,520,el granada,37.5027,-122.4694,23.89,69,0,2.06,US,1724610023


### Step 3: Create a new DataFrame called `hotel_df`.

In [46]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_temp_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = " "
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity","Hotel Name"]]

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
124,colonia,DE,50.9333,6.9500,50,
157,newman,US,37.3138,-121.0208,49,
241,laguna,US,38.4210,-121.4238,49,
259,toliara,MG,-23.3500,43.6667,67,
368,lewiston orchards,US,46.3804,-116.9754,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":f"accommodation.hotel",
    "apiKey": geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
#get latitude, longitude from the DataFrame
    latitude= hotel_df.loc[index,"Lat"]
    longitude=hotel_df.loc[index,"Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["lat"] = latitude
    params["lon"] = longitude
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
colonia - nearest hotel: Wasserturm Hotel Cologne
newman - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
toliara - nearest hotel: Ambary
lewiston orchards - nearest hotel: Hampton Inn Lewiston
pacifica - nearest hotel: Fairfield Inn & Suites
salinas - nearest hotel: The Traveler's Hotel
pacific grove - nearest hotel: Pacific Grove Inn
avenal - nearest hotel: No hotel found
el granada - nearest hotel: Beach House
diego de almagro - nearest hotel: Residencial Hambyll


,City,Country,Lat,Lng,Humidity,Hotel Name
124,colonia,DE,50.9333,6.9500,50,Wasserturm Hotel Cologne
157,newman,US,37.3138,-121.0208,49,No hotel found
241,laguna,US,38.4210,-121.4238,49,Holiday Inn Express & Suites
259,toliara,MG,-23.3500,43.6667,67,Ambary
368,lewiston orchards,US,46.3804,-116.9754,43,Hampton Inn Lewiston
370,pacifica,US,37.6138,-122.4869,75,Fairfield Inn & Suites
384,salinas,US,36.6777,-121.6555,50,The Traveler's Hotel
465,pacific grove,US,36.6177,-121.9166,68,Pacific Grove Inn
495,avenal,US,36.0041,-120.1290,36,No hotel found
520,el granada,US,37.5027,-122.4694,69,Beach House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
hotel_df.map = hotel_df.hvplot.points

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
hotel_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)